This lab on Logistic Regression in R comes from p. 154-161 of "Introduction to Statistical Learning with Applications in R" by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani

# 4.6.2 Logistic Regression

Let's return to the ${\tt Smarket}$ data from ${\tt ISLR}$. 

In [1]:
library(ISLR)
names(Smarket)
dim(Smarket)
summary(Smarket)

[1] "Year"      "Lag1"      "Lag2"      "Lag3"      "Lag4"      "Lag5"     
[7] "Volume"    "Today"     "Direction"

[1] 1250    9

      Year           Lag1                Lag2                Lag3          
 Min.   :2001   Min.   :-4.922000   Min.   :-4.922000   Min.   :-4.922000  
 1st Qu.:2002   1st Qu.:-0.639500   1st Qu.:-0.639500   1st Qu.:-0.640000  
 Median :2003   Median : 0.039000   Median : 0.039000   Median : 0.038500  
 Mean   :2003   Mean   : 0.003834   Mean   : 0.003919   Mean   : 0.001716  
 3rd Qu.:2004   3rd Qu.: 0.596750   3rd Qu.: 0.596750   3rd Qu.: 0.596750  
 Max.   :2005   Max.   : 5.733000   Max.   : 5.733000   Max.   : 5.733000  
      Lag4                Lag5              Volume           Today          
 Min.   :-4.922000   Min.   :-4.92200   Min.   :0.3561   Min.   :-4.922000  
 1st Qu.:-0.640000   1st Qu.:-0.64000   1st Qu.:1.2574   1st Qu.:-0.639500  
 Median : 0.038500   Median : 0.03850   Median :1.4229   Median : 0.038500  
 Mean   : 0.001636   Mean   : 0.00561   Mean   :1.4783   Mean   : 0.003138  
 3rd Qu.: 0.596750   3rd Qu.: 0.59700   3rd Qu.:1.6417   3rd Qu.: 0.596750  
 Max. 

In this lab, we will fit a logistic regression model in order to predict ${\tt Direction}$ using ${\tt Lag1}$ through ${\tt Lag5}$ and ${\tt Volume}$. The ${\tt glm()}$ function fits **generalized linear models**, a class of models that includes logistic regression. 

The syntax of the ${\tt glm()}$ function is similar to that of ${\tt lm()}$, except that we must pass in the argument ${\tt family=binomial}$ in order to tell ${\tt R}$ to run a logistic regression rather than some other type of generalized linear model.

In [2]:
glm.fit=glm(Direction~Lag1+Lag2+Lag3+Lag4+Lag5+Volume,data=Smarket,family=binomial)
summary(glm.fit)


Call:
glm(formula = Direction ~ Lag1 + Lag2 + Lag3 + Lag4 + Lag5 + 
    Volume, family = binomial, data = Smarket)

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-1.446  -1.203   1.065   1.145   1.326  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)
(Intercept) -0.126000   0.240736  -0.523    0.601
Lag1        -0.073074   0.050167  -1.457    0.145
Lag2        -0.042301   0.050086  -0.845    0.398
Lag3         0.011085   0.049939   0.222    0.824
Lag4         0.009359   0.049974   0.187    0.851
Lag5         0.010313   0.049511   0.208    0.835
Volume       0.135441   0.158360   0.855    0.392

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1731.2  on 1249  degrees of freedom
Residual deviance: 1727.6  on 1243  degrees of freedom
AIC: 1741.6

Number of Fisher Scoring iterations: 3


The smallest p-value here is associated with ${\tt Lag1}$. The negative coefficient
for this predictor suggests that if the market had a positive return yesterday,
then it is less likely to go up today. However, at a value of 0.145, the p-value
is still relatively large, and so there is no clear evidence of a real association
between ${\tt Lag1}$ and ${\tt Direction}$.

We use the ${\tt coef()}$ function in order to access just the coefficients for this
fitted model. We can also use the ${\tt summary()}$ function to access particular
aspects of the fitted model, such as the p-values for the coefficients.

In [ ]:
coef(glm.fit)
summary(glm.fit)$coef[,4] #p-values are stored in the 4th column

The ${\tt predict()}$ function can be used to predict the probability that the
market will go up, given values of the predictors. The ${\tt type="response"}$
option tells ${\tt R}$ to output probabilities of the form ${\tt P(Y = 1|X)}$, as opposed
to other information such as the ${\tt logit}$. If no data set is supplied to the
${\tt predict()}$ function, then the probabilities are computed for the training
data that was used to fit the logistic regression model. 

In [ ]:
glm.probs=predict(glm.fit,type="response")
glm.probs[1:10]

Here we have printed
only the first ten probabilities. We know that these values correspond to
the probability of the market going up, rather than down, because the
${\tt contrasts()}$ function indicates that ${\tt R}$ has created a dummy variable with
a 1 for ${\tt Up}$.

In [ ]:
contrasts(Smarket$Direction)

In order to make a prediction as to whether the market will go up or
down on a particular day, we must convert these predicted probabilities
into class labels, ${\tt Up}$ or ${\tt Down}$. The following two commands create a vector
of class predictions based on whether the predicted probability of a market
increase is greater than or less than 0.5.

In [ ]:
glm.pred=rep("Down",1250)
glm.pred[glm.probs>.5]="Up"

The first command creates a vector of 1,250 ${\tt Down}$ elements. The second line
transforms to ${\tt Up}$ all of the elements for which the predicted probability of a
market increase exceeds 0.5. Given these predictions, the ${\tt table()}$ function
can be used to produce a confusion matrix in order to determine how many
observations were correctly or incorrectly classified.

In [ ]:
table(glm.pred,Smarket$Direction)
mean(glm.pred==Smarket$Direction)

The diagonal elements of the confusion matrix indicate correct predictions,
while the off-diagonals represent incorrect predictions. Hence our model
correctly predicted that the market would go up on 507 days and that
it would go down on 145 days, for a total of 507 + 145 = 652 correct
predictions. The ${\tt mean()}$ function can be used to compute the fraction of
days for which the prediction was correct. In this case, logistic regression
correctly predicted the movement of the market 52.2% of the time.

At first glance, it appears that the logistic regression model is working
a little better than random guessing. But remember, this result is misleading
because we trained and tested the model on the same set of 1,250 observations.
In other words, 100− 52.2 = 47.8% is the **training error rate**. As we
have seen previously, the training error rate is often overly optimistic — it
tends to underestimate the _test_ error rate. 

In order to better assess the accuracy
of the logistic regression model in this setting, we can fit the model
using part of the data, and then examine how well it predicts the held out
data. This will yield a more realistic error rate, in the sense that in practice
we will be interested in our model’s performance not on the data that
we used to fit the model, but rather on days in the future for which the
market’s movements are unknown.

Like we did with KNN, we will first create a vector corresponding
to the observations from 2001 through 2004. We will then use this vector
to create a held out data set of observations from 2005.

In [ ]:
train=(Smarket$Year<2005)
Smarket.2005=Smarket[!train,]
Direction.2005=Smarket$Direction[!train]

We now fit a logistic regression model using only the subset of the observations
that correspond to dates before 2005, using the subset argument.
We then obtain predicted probabilities of the stock market going up for
each of the days in our test set—that is, for the days in 2005.

In [ ]:
glm.fit=glm(Direction~Lag1+Lag2+Lag3+Lag4+Lag5+Volume,data=Smarket,family=binomial,subset=train)
glm.probs=predict(glm.fit,Smarket.2005,type="response")

Notice that we have trained and tested our model on two completely separate
data sets: training was performed using only the dates before 2005,
and testing was performed using only the dates in 2005. Finally, we compute
the predictions for 2005 and compare them to the actual movements
of the market over that time period.

In [ ]:
glm.pred=rep("Down",252)
glm.pred[glm.probs>.5]="Up"
table(glm.pred,Direction.2005)
mean(glm.pred==Direction.2005)
mean(glm.pred!=Direction.2005)

The $!=$ notation means **not equal to**, and so the last command computes
the test set error rate. The results are rather disappointing: the test error
rate is 52%, which is worse than random guessing! Of course this result
is not all that surprising, given that one would not generally expect to be
able to use previous days’ returns to predict future market performance.
(After all, if it were possible to do so, then the authors of this book [along with your professor] would probably
be out striking it rich rather than teaching statistics.)

We recall that the logistic regression model had very underwhelming pvalues
associated with all of the predictors, and that the smallest p-value,
though not very small, corresponded to ${\tt Lag1}$. Perhaps by removing the
variables that appear not to be helpful in predicting ${\tt Direction}$, we can
obtain a more effective model. After all, using predictors that have no
relationship with the response tends to cause a deterioration in the test
error rate (since such predictors cause an increase in variance without a
corresponding decrease in bias), and so removing such predictors may in
turn yield an improvement. 

In the space below, refit a logistic regression using just ${\tt Lag1}$ and ${\tt Lag2}$, which seemed to have the highest predictive power in the original logistic regression model.

In [ ]:
glm.fit = # Write your code to fit the new model here

# This will test your new model
glm.probs=predict(glm.fit,Smarket.2005,type="response")
glm.pred=rep("Down",252)
glm.pred[glm.probs>.5]="Up"
table(glm.pred,Direction.2005)
mean(glm.pred==Direction.2005)

Now the results appear to be more promising: 56% of the daily movements
have been correctly predicted. The confusion matrix suggests that on days
when logistic regression predicts that the market will decline, it is only
correct 50% of the time. However, on days when it predicts an increase in
the market, it has a 58% accuracy rate.

Finally, suppose that we want to predict the returns associated with **particular
values** of ${\tt Lag1}$ and ${\tt Lag2}$. In particular, we want to predict Direction on a
day when ${\tt Lag1}$ and ${\tt Lag2}$ equal 1.2 and 1.1, respectively, and on a day when
they equal 1.5 and −0.8. We do this using the ${\tt predict()}$ function.

In [ ]:
predict(glm.fit,newdata=data.frame(Lag1=c(1.2,1.5),Lag2=c(1.1,-0.8)),type="response")

To get credit for this lab, play around with a few other values for ${\tt Lag1}$ and ${\tt Lag2}$, and then post to Piazza about what you found. If you're feeling adventurous, try fitting models with other subsets of variables to see if you can find a letter one!